In [6]:
%load_ext rpy2.ipython

import pandas as pd
import networkx as nx
import warnings
import numpy as np
import microbe_directory as md
import seaborn as sns

from matplotlib import pyplot as plt        
import matplotlib.patches as mpatches


from capalyzer.packet_parser import DataTableFactory, NCBITaxaTree, annotate_taxa, TaxaTree
from capalyzer.packet_parser.data_utils import group_small_cols
from capalyzer.packet_parser.diversity_metrics import (
    shannon_entropy, richness, chao1, rarefaction_analysis
)
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, cophenet, leaves_list
from scipy.spatial.distance import squareform, pdist, jensenshannon

from os.path import join
from metasub_utils.packet_parse import MetaSUBTableFactory
from metasub_utils.packet_parse.metadata_ontology import add_ontology
from capalyzer.packet_parser.experimental import umap
from capalyzer.packet_parser.data_utils import group_small_cols
from capalyzer.packet_parser.normalize import proportions, prevalence
from plotnine import *
from scipy.cluster.hierarchy import fcluster
from matplotlib import pyplot as plt

from capalyzer.constants import MICROBE_DIR

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm 
from sklearn.metrics import (
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
    accuracy_score,
)

warnings.filterwarnings('ignore')

NAN = float('nan')
PACKET_DIR = '/home/dcdanko/Dropbox/resources_and_shared/metasub_data_packets/release_packet_jun12_2019/'
PACKET_DIR = '/home/dcdanko/Dropbox/resources_and_shared/metasub_data_packets/data_packet_2020_02_23_16_12'
ncbi_tree = NCBITaxaTree.parse_files()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [9]:
def to_title(el):
    sel = str(el)
    if not el:
        return el
    tkns = sel.split('_')
    tkns = [tkn[0].upper() + tkn[1:] for tkn in tkns if tkn]
    return ' '.join(tkns)


def categorical_continents(continents):
    return pd.Categorical(
        pd.Series(continents).map(to_title),
        categories=[
            'North America',
            'East Asia',
            'Europe',
            'Sub Saharan Africa',
            'South America',
            'Middle East',
            'Oceania',
            'Nan',
        ],
        ordered=True,
    )

def normalize_surface(el):
    try:
        el = el.lower()
    except AttributeError:
        return el
    el = '_'.join(el.split())
    return {
         'ground': 'floor',
         'palm_left': 'human_hand',
         'vert_pole': 'pole',
         'palm_right': 'human_hand',
         'pedestrian_crossing_button': 'button',
         'wooden bench': 'seat',
         'gargabe_can': 'garbage',
         'horiz_pole': 'railing',
         'stairwell railing': 'railing',
         'bike kiosk': 'kiosk',
         'bench1': 'seat',
         'kiosk2': 'kiosk',
         'kiosk1': 'kiosk',
         'ticket_machine': 'kiosk',
         'escalator_handrail': 'railing',
         'bench2': 'seat',
         'lift_buttons': 'button',
         'overhead_handrail': 'railing',
         'poll': 'pole',
        'garbage_can': 'garbage',
        'bench': 'seat',
        'door_handle;door_handle': 'door_knob',
        'glass': 'window',
        'ceiling_rail': 'railing',
        'seat_rail': 'railing',
        'ticket_hall;ticket_machine': 'kiosk',
        'bench;platform': 'seat',
        'platform;bench': 'seat',
        'ticket_machine;ticket_hall': 'kiosk',
        'wooden_bench': 'seat',
        'stairwell_railing': 'railing',
        'trolley_handle': '',
        'turnstile_or_alternatives;turnstile_or_alternatives': 'turnstile',
        'ticket_kiosks;ticket_kiosks': 'kiosk',
        'ticket_machine;ticket_machine': 'kiosk',
        'station_eletronic_kiosk': 'kiosk',
        'station_railing': 'railing',
        'station_seat': 'seat',
        'center_seat': 'seat',
        'negative_control_(air)': '',
        'ceiling_rail;ceiling_rail': 'railing',
        'rail': 'railing',
        'seat_near_door': 'seat',
        'ticketing_machine;ticketing_machine_': 'kiosk',
        'jetway_1_seats_waiting_area': 'seat',
    }.get(el, el)

def add_ontology(metadata):
    """Return a pandas dataframe with metadata and MetaSUB ontologies."""
    metadata = add_surface_ontology(metadata)
    metadata = add_place_ontology(metadata)
    return metadata


def has_keyword(target, *queries):
    try:
        for query in queries:
            if query in target:
                return True
    except TypeError:
        pass
    return False


def add_surface_ontology(metadata):
    """Return a pandas dataframe with metadata and surface ontologies."""
    metadata, tbl = metadata.copy(), {}
    for val in metadata['surface_material'].unique():
        if has_keyword(val, 'glass', 'metal', 'steel', 'copper'):
            tbl[val] = ('metal', 'impermeable')
        elif has_keyword(val, 'stone', 'marble', 'ceramic', 'concrete', 'cement', 'granite'):
            tbl[val] = ('stone', 'impermeable')
        elif has_keyword(val, 'plastic', 'rubber', 'vinyl', 'pvc', 'formica'):
            tbl[val] = ('plastic', 'impermeable')
        elif has_keyword(val, 'fabric', 'cloth', 'carpet'):
            tbl[val] = ('fabric', 'permeable')
        elif has_keyword(val, 'hand', 'flesh', 'wood', 'leather', 'fiber'):
            tbl[val] = ('biological', 'permeable')
        elif has_keyword(val, 'control'):
            tbl[val] = ('control', 'control')
        else:
            tbl[val] = (NAN, NAN)
    metadata['surface_ontology_fine'] = metadata['surface_material'].apply(lambda x: tbl[x][0])
    metadata['surface_ontology_coarse'] = metadata['surface_material'].apply(lambda x: tbl[x][1])
    return metadata

def add_place_ontology(metadata):
    """Return a pandas dataframe with metadata and place ontologies."""
    metadata = metadata.copy()
    metadata['coastal'] = metadata.apply(coastal, result_type='reduce', axis=1)
    return metadata


def coastal(row):
    if row['coastal_city'] == 'yes':
        return ('coastal', 'coastal')
    if float(row['city_elevation_meters']) > 1000:
        return ('high altitude', 'not_coastal')
    return ('low altitude', 'not_coastal')

In [10]:
class MetaSUBFiguresData:

    def __init__(self, packet_dir, ncbi_tree=None):
        self.tabler = DataTableFactory(packet_dir, metadata_tbl='metadata/complete_metadata.csv')
        print(self.tabler.metadata.shape)
        self.tabler.metadata = add_ontology(self.tabler.metadata)
        self.tabler.metadata['city_elevation'] = self.tabler.metadata['coastal'].map(lambda el: el[1])
        self.tabler.metadata['coastal'] = self.tabler.metadata['coastal'].map(lambda el: el[0])
        self.tabler.metadata['continent'] = categorical_continents(self.tabler.metadata['continent'])
        self.tabler.metadata['city'] = self.tabler.metadata['city'].map(to_title)
        no_air = self.tabler.metadata.query('project != "CSD17_AIR"')
        self.tabler = self.tabler.copy(new_metadata=no_air)
        self.meta = self.tabler.metadata
        self._long_taxa = None
        self._wide_taxa = None

    @property
    def amr_genes(self):
        return self.build_amr_genes()

    @property
    def long_taxa(self):
        if self._long_taxa is None:
            self._long_taxa = self.build_long_taxa()
        return self._long_taxa

    @property
    def wide_taxa(self):
        if self._wide_taxa is None:
            self._wide_taxa = self.build_wide_taxonomy()
        return self._wide_taxa

    @property
    def wide_taxa_rel(self):
        return proportions(self.wide_taxa)
    
    def build_amr_genes(self):
        """TODO: REVIEW."""
        amrs = self.tabler.amrs(kind='gene', remove_zero_rows=False).dropna()
        amrs = (amrs.T / (amrs.T.sum() + 0.000001)).T
        return amrs
        
    def build_wide_taxonomy(self):
        """Return a pandas df with species in columns, samples in rows. Values are read counts."""
        # return self.tabler.taxonomy()
        return self.long_taxa.pivot(index='sample', columns='taxa_name', values='reads').fillna(0)

    def build_long_taxa(self):
        long_taxa = pd.read_csv(join(self.tabler.packet_dir, 'taxonomy/refseq.krakenhll_longform.csv.gz'))
        long_taxa = long_taxa.rename(columns={
            'Unnamed: 0': 'sample',
            'Unnamed: 1': 'taxa_name',
            'Unnamed: 2': 'taxa_id',
            'Unnamed: 3': 'rank',
            'Unnamed: 4': 'mpa',
        })
        sample_list = self.tabler.metadata.index
        long_taxa = long_taxa.loc[long_taxa['sample'].isin(sample_list)]
        long_taxa = long_taxa.query('rank == "species"')
        for col in ['reads', 'kmers', 'tax_reads']:
            long_taxa[col] = long_taxa[col].fillna(0).map(int)
        for col in ['dup', 'cov', 'percent']:
            long_taxa[col] = long_taxa[col].fillna(0).map(float)

        long_taxa['kmer_read_ratio'] = long_taxa['kmers'] / long_taxa['reads']

        long_taxa = long_taxa.query('kmer_read_ratio >= 2.5')
        long_taxa = long_taxa.query('kmers >= 64')
        long_taxa = long_taxa.query('reads >= 3')
        long_taxa = long_taxa.query('taxa_name != "Human endogenous retrovirus"')
        long_taxa = long_taxa.query('taxa_name != "Human endogenous retrovirus K"')
        long_taxa = long_taxa.loc[long_taxa['sample'].isin(self.meta.index)]
        return long_taxa
        
data = MetaSUBFiguresData(PACKET_DIR)


(5132, 41)


In [11]:
meta = data.meta
taxa = data.wide_taxa.fillna(0)
amrs = data.amr_genes
meta.head()

,metasub_name,core_project,project,city,city_code,latitude,longitude,surface_material,control_type,elevation,...,index_sequence,location_type,hudson_alpha_uid,other_project_uid,sample_type,sl_name,surface_ontology_fine,surface_ontology_coarse,coastal,city_elevation
uuid,,,,,,,,,,,,,,,,,,,,,
haib17KIU4866_H7HJMCCXY_SL272833,CSD16-LIS-CTRL-AIRPORT,core,CSD16,Lisbon,LIS,NaN,NaN,NaN,NaN,NaN,...,CATCCGA,NaN,haib17KIU4866_H7HJMCCXY_SL272833,NaN,NaN,SL272833,NaN,NaN,coastal,coastal
haib17KIU4866_H7HJMCCXY_SL272830,CSD16-LIS-CTRL-LINEA,core,CSD16,Lisbon,LIS,NaN,NaN,NaN,NaN,NaN,...,GTATACA,NaN,haib17KIU4866_H7HJMCCXY_SL272830,NaN,NaN,SL272830,NaN,NaN,coastal,coastal
haib17KIU4866_H7HJMCCXY_SL272831,CSD16-LIS-CTRL-LINEC,core,CSD16,Lisbon,LIS,NaN,NaN,NaN,NaN,NaN,...,AGATCCC,NaN,haib17KIU4866_H7HJMCCXY_SL272831,NaN,NaN,SL272831,NaN,NaN,coastal,coastal
haib17KIU4866_H7HJMCCXY_SL272832,CSD16-LIS-CTRL-LINED,core,CSD16,Lisbon,LIS,NaN,NaN,NaN,NaN,NaN,...,GCCGTAT,NaN,haib17KIU4866_H7HJMCCXY_SL272832,NaN,NaN,SL272832,NaN,NaN,coastal,coastal
haib17KIU4866_HTWH2CCXY_SL364012,CSD16-MRS-0047,core,CSD16,Marseille,MRS,NaN,NaN,NaN,NaN,NaN,...,AATACGAA-AGGCCTGC,NaN,haib17KIU4866_HTWH2CCXY_SL364012,NaN,NaN,SL364012,NaN,NaN,coastal,coastal


In [12]:
taxa.head()

taxa_name,Acanthamoeba polyphaga mimivirus,Acaryochloris marina,Acetoanaerobium sticklandii,Acetobacter aceti,Acetobacter pasteurianus,Acetobacter persici,Acetobacter pomorum,Acetobacter senegalensis,Acetobacter sp. SLV-7,Acetobacter tropicalis,...,endosymbiont of Acanthamoeba sp. UWC8,endosymbiont of unidentified scaly snail isolate Monju,gamma proteobacterium HdN1,halophilic archaeon DL31,halophilic archaeon True-ADL,methanogenic archaeon ISO4-H5,secondary endosymbiont of Ctenarytaina eucalypti,secondary endosymbiont of Heteropsylla cubana,secondary endosymbiont of Trabutina mannipara,uncultured crAssphage
sample,,,,,,,,,,,,,,,,,,,,,
haib17CEM4890_H2NYMCCXY_SL254769,0.0,0.0,0.0,5.0,28.0,0.0,0.0,0.0,0.0,0.0,...,0.0,52.0,0.0,0.0,0.0,0.0,0.0,25.0,14.0,0.0
haib17CEM4890_H2NYMCCXY_SL254770,0.0,0.0,0.0,38.0,133.0,0.0,0.0,5.0,0.0,0.0,...,0.0,37.0,0.0,0.0,0.0,0.0,0.0,32.0,21.0,0.0
haib17CEM4890_H2NYMCCXY_SL254771,0.0,0.0,0.0,48.0,339.0,16.0,0.0,0.0,0.0,28.0,...,0.0,87.0,440.0,0.0,0.0,0.0,0.0,13.0,8.0,0.0
haib17CEM4890_H2NYMCCXY_SL254772,0.0,0.0,0.0,24.0,69.0,0.0,0.0,5.0,0.0,0.0,...,14.0,0.0,30.0,0.0,0.0,0.0,0.0,6.0,6.0,0.0
haib17CEM4890_H2NYMCCXY_SL254773,0.0,0.0,28.0,30.0,85.0,25.0,3.0,19.0,0.0,29.0,...,0.0,45.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,118.0


In [13]:
design = meta.loc[taxa.index, ['city_koppen_climate', 'setting', 'elevation']].fillna('unknown')
#design['city_population_density'] = design['city_population_density'].map(lambda x: float(x) if x != 'unknown' else float('nan'))
#design['continent'] = [str(el) for el in design['continent']]
#design['surface_ontology_fine'] = [str(el) for el in design['surface_ontology_fine']]
#design = design.query('continent != "0"').query('city_population_density > 0')
taxa = taxa.loc[design.index]

taxa_prev = (taxa > 0).mean()
taxa = taxa[taxa_prev[taxa_prev > 0.25].index]
taxa += 1

print(taxa.shape)
print(design.shape)
design.head()

(4531, 1952)
(4531, 3)


,city_koppen_climate,setting,elevation
sample,,,
haib17CEM4890_H2NYMCCXY_SL254769,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254770,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254771,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254772,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254773,humid_subtropical_climate,unknown,unknown


In [14]:
%%R -i taxa -i design

devtools::load_all('/home/dcdanko/Dev/mavric/')

mav = estVC(t(taxa), design, clist, verbose=T, autosel=F, sigcor=F)

R[write to console]: Loading mavric



[1] "Confirmed inputs"
[1] "running HTS"


R[write to console]: converting counts to integer mode

R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



[1] "Built PCA Obj"
[1] "Finding relevant PCs"


In [15]:
%%R
        
design$city_koppen_climate = as.factor(design$city_koppen_climate)
design$setting = as.factor(design$setting)
design$elevation = as.factor(design$elevation)

plotVars(mav, design, plotve=F)

                                      setting\n2% 
                                       0.00418853 
                                    elevation\n2% 
                                       0.03722007 
              city_koppen_climate\n6%&setting\n2% 
                                       0.04271508 
city_koppen_climate\n6%&elevation\n2%&setting\n2% 
                                       0.55862373 
                        elevation\n2%&setting\n2% 
                                       1.07463200 
                          city_koppen_climate\n6% 
                                       4.91026690 
                                   Discarded\n47% 
                                      46.56743136 
                                 Unexplained\n47% 
                                      46.80492232 


In [16]:
list(meta.columns)

['metasub_name',
 'core_project',
 'project',
 'city',
 'city_code',
 'latitude',
 'longitude',
 'surface_material',
 'control_type',
 'elevation',
 'line',
 'station',
 'surface',
 'temperature',
 'setting',
 'num_reads',
 'library_post_PCR_Qubit',
 'library_QC_concentration',
 'city_latitude',
 'city_longitude',
 'coastal_city',
 'city_total_population',
 'city_population_density',
 'city_land_area_km2',
 'city_ave_june_temp_c',
 'city_elevation_meters',
 'continent',
 'city_koppen_climate',
 'barcode',
 'ha_id',
 'hudson_alpha_flowcell',
 'hudson_alpha_project',
 'index_sequence',
 'location_type',
 'hudson_alpha_uid',
 'other_project_uid',
 'sample_type',
 'sl_name',
 'surface_ontology_fine',
 'surface_ontology_coarse',
 'coastal',
 'city_elevation']

In [26]:
amr_snames = amrs.index.to_list()
amrs2 = (1000 * 1000 * amrs).applymap(int)
amr_design =design.loc[amr_snames]

In [27]:
%%R -i amrs2 -i amr_design

devtools::load_all('/home/dcdanko/Dev/mavric/')

mav_amr = estVC(t(amrs2), amr_design, clist, verbose=T, autosel=F, sigcor=F)

R[write to console]: Loading mavric



[1] "Confirmed inputs"
[1] "running HTS"


R[write to console]: Error in estimateSizeFactorsForMatrix(counts(object), locfunc = locfunc,  : 
  every gene contains at least one zero, cannot compute log geometric means
Calls: <Anonymous> ... estimateSizeFactors -> .local -> estimateSizeFactorsForMatrix




Error in estimateSizeFactorsForMatrix(counts(object), locfunc = locfunc,  : 
  every gene contains at least one zero, cannot compute log geometric means
Calls: <Anonymous> ... estimateSizeFactors -> .local -> estimateSizeFactorsForMatrix


TypeError: int() argument must be a string, a bytes-like object or a number, not 'DataFrame'

In [20]:
design

,city_koppen_climate,setting,elevation
sample,,,
haib17CEM4890_H2NYMCCXY_SL254769,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254770,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254771,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254772,tropical_savanna_climate,unknown,unknown
haib17CEM4890_H2NYMCCXY_SL254773,humid_subtropical_climate,unknown,unknown
...,...,...,...
sossowski_BarcelonaNov2018_CSD16-BCN-258-29786-GGACTCCT-TATCCTCT,hot-summer_mediterranean_climate,unknown,unknown
sossowski_BarcelonaNov2018_CSD16-BCN-259-29787-GGACTCCT-GTAAGGAG,hot-summer_mediterranean_climate,unknown,unknown
sossowski_BarcelonaNov2018_CSD16-BCN-261-29787-GGACTCCT-AAGGAGTA,hot-summer_mediterranean_climate,unknown,unknown


In [ ]:
cols = [
    'city',
    'surface',
    'surface_ontology_fine',
    'city_koppen_climate',
    'setting',
    'elevation',
    'coastal',
    'hudson_alpha_flowcell',
    'continent',
]


design = meta.loc[taxa.index, cols]

for col in cols:
    if col == 'continent':
        continue
    design[col] = design[col].fillna('unknown')

def normalize_surface(el):
    try:
        el = el.lower()
    except AttributeError:
        return el
    el = '_'.join(el.split())
    return {
         'ground': 'floor',
         'palm_left': 'human_hand',
         'vert_pole': 'pole',
         'palm_right': 'human_hand',
         'pedestrian_crossing_button': 'button',
         'wooden bench': 'seat',
         'gargabe_can': 'garbage',
         'horiz_pole': 'railing',
         'stairwell railing': 'railing',
         'bike kiosk': 'kiosk',
         'bench1': 'seat',
         'kiosk2': 'kiosk',
         'kiosk1': 'kiosk',
         'ticket_machine': 'kiosk',
         'escalator_handrail': 'railing',
         'bench2': 'seat',
         'lift_buttons': 'button',
         'overhead_handrail': 'railing',
         'poll': 'pole',
        'garbage_can': 'garbage',
        'bench': 'seat',
        'door_handle;door_handle': 'door_knob',
        'glass': 'window',
        'ceiling_rail': 'railing',
        'seat_rail': 'railing',
        'ticket_hall;ticket_machine': 'kiosk',
        'bench;platform': 'seat',
        'platform;bench': 'seat',
        'ticket_machine;ticket_hall': 'kiosk',
        'wooden_bench': 'seat',
        'stairwell_railing': 'railing',
        'trolley_handle': '',
        'turnstile_or_alternatives;turnstile_or_alternatives': 'turnstile',
        'ticket_kiosks;ticket_kiosks': 'kiosk',
        'ticket_machine;ticket_machine': 'kiosk',
        'station_eletronic_kiosk': 'kiosk',
        'station_railing': 'railing',
        'station_seat': 'seat',
        'center_seat': 'seat',
        'negative_control_(air)': '',
        'ceiling_rail;ceiling_rail': 'railing',
        'rail': 'railing',
        'seat_near_door': 'seat',
        'ticketing_machine;ticketing_machine_': 'kiosk',
        'jetway_1_seats_waiting_area': 'seat',
    }.get(el, el)

design['surface'] = design['surface'].map(normalize_surface)
n_samps = design['surface'].value_counts()
allowed_surfaces = set(n_samps[n_samps >= 8].index)
design['surface'] = design['surface'].map(lambda el: 'other' if el not in allowed_surfaces else el)

design = design.query('continent != "Nan"')
design['continent'] = design['continent'].map(str)
design = design.dropna()
taxa = taxa.loc[design.index]

taxa_prev = (taxa > 0).mean()
taxa = taxa[taxa_prev[taxa_prev > 0.25].index]
taxa += 1

print(taxa.shape)
print(design.shape)
design.head()

In [ ]:
design['continent'].unique()

In [ ]:
%%R -i taxa -i design

devtools::load_all('/home/dcdanko/Dev/mavric/')

mav = estVC(t(taxa), design, clist, verbose=T, autosel=F, sigcor=F)

In [ ]:
%%R -i design

attributes(mav)

In [ ]:
%%R

mav$pcavar

In [ ]:
%%R

plotVars <- function(results, annotation, plotzero = TRUE, incvals = FALSE, plotve = TRUE, maxStress = .1) {
    results$pcs <- lapply(
        results$pcs,
        function(e) e[as.numeric(rownames(e)) > 0, , drop = F]
    )
    print(10)
    vars <- round(
        unlist(
            sapply(1:length(results$pcs), function(p) {
                if(nrow(results$pcs[[p]]) == 0) return(0)
                return(
                    sum(
                        results$pcavar[as.numeric(rownames(results$pcs[[p]]))] *
                        sapply(
                            as.numeric(rownames(results$pcs[[p]])),
                            function(i) totVarEx(results$pcasp, annotation, i, p)
                        )
                    )
                )
            })
        )
    )
    print(20)
    vn <- paste(colnames(annotation), paste(vars, "%", sep=''), sep='\n')
    print(vn)
    print(21)
    pcex <- sort(unique(unlist(lapply(results$pcs, rownames))))
    print(22)
    matrixed <- matrix(
        unlist(
            sapply(1:length(results$pcs), function(li) {
                rv <- rep(0, length(pcex))
                if(nrow(results$pcs[[li]]) > 0) {
                    e <- rownames(results$pcs[[li]])
                    for(i in 1:length(e)) rv[match(e[i], pcex)] <- results$pcavar[as.numeric(e[i])]*totVarEx(results$pcasp, annotation, as.numeric(e[i]), li)
                }
                return(rv)
            })
        ),
        nrow=length(pcex),
        dimnames=list(pcex, vn)
    )
    print(25)
    applied <- apply(
        matrixed,
        1,
        function(i) {
            print(i)
            print(25.1)
            myorder <- order(i, decreasing=T)
            print(25.15)
            myn = sum((i > 0) & !is.na(i))
            print(25.18)
            print(myn)
            print(myorder)
            nz <- head(myorder, n=myn)
            print(25.2)
            if(length(nz) == 1) return(rbind(i[nz], vn[nz]))
            print(25.3)
            return(rbind(c(diff(t(embed(i[nz], 2))), i[nz[length(nz)]]), sapply(1:length(nz), function(n) paste(vn[nz[1:n]], collapse='&'))))
        }
    )
    print(27)
    unlisted <- unlist(applied)
    print(28)
    left_matrix <- matrix(unlisted, nrow=2)
    print(29)
    right_matrix <- matrix(
        sapply(
            unlist(
                sapply(1:length(results$pcs), function(li) if(nrow(results$pcs[[li]]) == 0) return(c(0, vn[li])))
            ),
            function(i) i
        ),
        nrow=2
    )
    pcm <- cbind(left_matrix, right_matrix)
    print(30)
    unacc <- 100-sum(results$pcavar)
    unex <- 100-(unacc+sum(as.numeric(pcm[1,])))
    other.var <- c(unex, unacc)
    names(other.var) <- paste(c("Unexplained", "Discarded"), paste(as.character(round(c(unex, unacc))), "%", sep=''), sep='\n')
    pl.vars <- sort(c(tapply(as.numeric(unlist(pcm[1,])), unlist(lapply(lapply(strsplit(unlist(pcm[2,]), "&"), sort), paste, collapse = '&')), sum), other.var), decreasing = incvals)
    if(!plotzero) pl.vars <- pl.vars[pl.vars > 0]
    if(!plotve) return(pl.vars)
    pl.vars.c <- ceiling(pl.vars)
    repeat {
        pl.tmp <- abs(jitter(pl.vars.c))
        if(plotzero) pl.tmp[pl.vars.c == 0] = 0
        pl.ve <- venneuler::venneuler(pl.tmp)
        if(pl.ve$stress < maxStress) break
    }
    print(40)
    if(plotve) plot(venneuler::venneuler(pl.tmp))
    return(pl.vars)
}

In [ ]:
%%R -i design

design$city_koppen_climate = as.factor(design$city_koppen_climate)
design$setting = as.factor(design$setting)
design$elevation = as.factor(design$elevation)
design$city = as.factor(design$city)
design$surface = as.factor(design$surface)
design$surface_ontology_fine = as.factor(design$surface_ontology_fine)
design$coastal = as.factor(design$coastal)
design$hudson_alpha_flowcell = as.factor(design$hudson_alpha_flowcell)
#design$continent = as.factor(design$continent)

plotVars(mav, design, plotve=F)

In [ ]:
cols = [
    'city',
    'surface',
    'surface_ontology_fine',
    'city_koppen_climate',
    'setting',
    'elevation',
    'coastal',
    'hudson_alpha_flowcell',
]


design = meta.loc[taxa.index, cols]

for col in cols:
    if col == 'continent':
        continue
    design[col] = design[col].fillna('unknown')

def normalize_surface(el):
    try:
        el = el.lower()
    except AttributeError:
        return el
    el = '_'.join(el.split())
    return {
         'ground': 'floor',
         'palm_left': 'human_hand',
         'vert_pole': 'pole',
         'palm_right': 'human_hand',
         'pedestrian_crossing_button': 'button',
         'wooden bench': 'seat',
         'gargabe_can': 'garbage',
         'horiz_pole': 'railing',
         'stairwell railing': 'railing',
         'bike kiosk': 'kiosk',
         'bench1': 'seat',
         'kiosk2': 'kiosk',
         'kiosk1': 'kiosk',
         'ticket_machine': 'kiosk',
         'escalator_handrail': 'railing',
         'bench2': 'seat',
         'lift_buttons': 'button',
         'overhead_handrail': 'railing',
         'poll': 'pole',
        'garbage_can': 'garbage',
        'bench': 'seat',
        'door_handle;door_handle': 'door_knob',
        'glass': 'window',
        'ceiling_rail': 'railing',
        'seat_rail': 'railing',
        'ticket_hall;ticket_machine': 'kiosk',
        'bench;platform': 'seat',
        'platform;bench': 'seat',
        'ticket_machine;ticket_hall': 'kiosk',
        'wooden_bench': 'seat',
        'stairwell_railing': 'railing',
        'trolley_handle': '',
        'turnstile_or_alternatives;turnstile_or_alternatives': 'turnstile',
        'ticket_kiosks;ticket_kiosks': 'kiosk',
        'ticket_machine;ticket_machine': 'kiosk',
        'station_eletronic_kiosk': 'kiosk',
        'station_railing': 'railing',
        'station_seat': 'seat',
        'center_seat': 'seat',
        'negative_control_(air)': '',
        'ceiling_rail;ceiling_rail': 'railing',
        'rail': 'railing',
        'seat_near_door': 'seat',
        'ticketing_machine;ticketing_machine_': 'kiosk',
        'jetway_1_seats_waiting_area': 'seat',
    }.get(el, el)

design['surface'] = design['surface'].map(normalize_surface)
n_samps = design['surface'].value_counts()
allowed_surfaces = set(n_samps[n_samps >= 8].index)
design['surface'] = design['surface'].map(lambda el: 'other' if el not in allowed_surfaces else el)



taxa = taxa.loc[design.index]

taxa_prev = (taxa > 0).mean()
taxa = taxa[taxa_prev[taxa_prev > 0.25].index]
taxa += 1

print(taxa.shape)
print(design.shape)
design.head()

In [ ]:
%%R -i taxa -i design

devtools::load_all('/home/dcdanko/Dev/mavric/')

mav = estVC(t(taxa), design, clist, verbose=T, autosel=F, sigcor=F)

In [ ]:
%%R

design$city_koppen_climate = as.factor(design$city_koppen_climate)
design$setting = as.factor(design$setting)
design$elevation = as.factor(design$elevation)
design$city = as.factor(design$city)
design$surface = as.factor(design$surface)
design$surface_ontology_fine = as.factor(design$surface_ontology_fine)
design$coastal = as.factor(design$coastal)
design$hudson_alpha_flowcell = as.factor(design$hudson_alpha_flowcell)

plotVars(mav, design, plotve=F)